## Scraping Multiple Pages - Indianapolis

#### Importing Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.parse
import pandas as pd

#### Scraping the data

In [2]:
address = []
bedrooms = []
bathrooms = []
sqft = []
year_built = []
parking = []
pool = []
price = []

# Url part I
url_1 = 'https://www.trulia.com'

for i in range(1, 26):
    
    #website
    www = 'https://www.trulia.com/IN/Indianapolis/' +str(i) +'_p/'
    
    #requests
    response = requests.get(www)
    
    # soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    #result container
    result_container = soup.find_all('li', {'class':'SearchResultsList__WideCell-b7y9ki-2'})
    
    #update results
    results_update = []
    
    # only results with attribute 'data-testeid'
    for r in result_container:
        if r.has_attr('data-testid'):
            results_update.append(r)
        
    #relative url
    relative_url = []
    
    #loop through results
    for i in results_update:
        for j in i.find_all('div', {'data-testid':'property-card-details'}):
            relative_url.append(j.find('a').get('href'))
            
    #empty list url_joined
    url_joined = []

    for i in relative_url:
        url_joined.append(urllib.parse.urljoin(url_1, i))
        
       
    # loop through all joined links
    for i in url_joined:
        response = requests.get(i)

        #create soup object
        soup = BeautifulSoup(response.content, 'html.parser')

        #address
        try:
            address.append(soup.find('span', {'data-testid':'home-details-summary-headline'}).get_text())
        except:
            address.append('')

        #bedrooms
        try:
            bedrooms.append(soup.find('li', {'data-testid':'bed'}).get_text())
        except:
            bedrooms.append('')

        #bathrooms
        try:
            bathrooms.append(soup.find('li', {'data-testid':'bath'}).get_text())
        except:
            bathrooms.append('')

        #sqft
        try:
            sqft.append(soup.find('li', {'data-testid':'floor'}).get_text())
        except:
            sqft.append('')

        #year built
        try:
            year_built.append(soup.find('div', string='Year Built').findNext('div').get_text())
        except:
            year_built.append('')

         #parking
        try:
            parking.append(soup.find('div', string='Parking').findNext('div').get_text())
        except:
            parking.append('')

         #pool
        try:
            pool.append(soup.find('div', string='Pool').findNext('div').get_text())
        except:
            pool.append('')

         #price
        try:
            price.append(soup.find('h3', {'data-testid':'on-market-price-details'}).get_text())
        except:
            price.append('')

        #creating a dictionary with the results
        output = {'Address': address, 'Bedrooms':bedrooms, 'Bathrooms': bathrooms, 'Area': sqft, 'Year Built': year_built,
                 'Parking': parking, 'Pool': pool, 'Price': price}

In [3]:
df = pd.DataFrame(output)
df

,Address,Bedrooms,Bathrooms,Area,Year Built,Parking,Pool,Price
0,6620 Olive Branch Ct,4 Beds,3 Baths,"2,350 sqft (on 0.29 acres)",2002,Garage,No,"$260,000"
1,5335 Thornleigh Dr,3 Beds,2 Baths,"1,599 sqft (on 0.48 acres)",1958,Garage,No,"$200,000"
2,3766 E 65th St,3 Beds,4 Baths,"2,936 sqft (on 0.42 acres)",1992,Garage,No,"$359,000"
3,526 S Pine St,3 Beds,3 Baths,"2,890 sqft",1887,No Info,No,"$340,000"
4,5543 Vicksburg Dr,4 Beds,3 Baths,"1,800 sqft",1989,Garage,No,"$209,900"
...,...,...,...,...,...,...,...,...
995,350 N Meridian St #802,4 Beds,4 Baths,"3,157 sqft",1924,Garage,,"$1,125,000"
996,1731 E Legrande Ave,3 Beds,1 Bath,"1,488 sqft",1947,Garage,No,"$119,000"
997,1106 Newman St,3 Beds,3 Baths,"2,568 sqft",1929,Garage,No,"$415,000"
998,1802 Woodlawn Ave,5 Beds,4 Baths,,1900,Open Parking,,"$245,000"


In [4]:
df['Location'] = 'Indianapolis'
df

,Address,Bedrooms,Bathrooms,Area,Year Built,Parking,Pool,Price,Location
0,6620 Olive Branch Ct,4 Beds,3 Baths,"2,350 sqft (on 0.29 acres)",2002,Garage,No,"$260,000",Indianapolis
1,5335 Thornleigh Dr,3 Beds,2 Baths,"1,599 sqft (on 0.48 acres)",1958,Garage,No,"$200,000",Indianapolis
2,3766 E 65th St,3 Beds,4 Baths,"2,936 sqft (on 0.42 acres)",1992,Garage,No,"$359,000",Indianapolis
3,526 S Pine St,3 Beds,3 Baths,"2,890 sqft",1887,No Info,No,"$340,000",Indianapolis
4,5543 Vicksburg Dr,4 Beds,3 Baths,"1,800 sqft",1989,Garage,No,"$209,900",Indianapolis
...,...,...,...,...,...,...,...,...,...
995,350 N Meridian St #802,4 Beds,4 Baths,"3,157 sqft",1924,Garage,,"$1,125,000",Indianapolis
996,1731 E Legrande Ave,3 Beds,1 Bath,"1,488 sqft",1947,Garage,No,"$119,000",Indianapolis
997,1106 Newman St,3 Beds,3 Baths,"2,568 sqft",1929,Garage,No,"$415,000",Indianapolis
998,1802 Woodlawn Ave,5 Beds,4 Baths,,1900,Open Parking,,"$245,000",Indianapolis


In [5]:
df.to_csv('df_indianapolis.csv', index=False)